In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

Load anndata

In [2]:
adata = sc.read_h5ad("test.h5ad")

In [3]:
# # lognorm should be the default (expected)
# adata.X = adata.layers['logcounts']

Parameters

In [4]:
groupby='label'
de_method='wilcoxon'
resource_name = 'consensus'
complex_policy = 'min'
key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']
verbose = False
base=2.718281828459045

In [5]:
# only relevant if perms are done
n_perms = 1000
seed = 69

Run LIANA PIPE

In [22]:
from liana import liana_pipe

In [28]:
lr_res = liana_pipe(adata=adata, 
                    groupby=groupby, 
                    resource_name=resource_name,
                    expr_prop=0,
                    de_method=de_method,
                    base=base,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    supp_cols=None,
                    resource=None,
                    use_raw=False,
                    layer='logcounts', 
                    _return_subunits=True
                   )

/home/dbdimitrov/Repos/liana-py/liana/steady/liana_pipe.py:67: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['label'] = adata.obs[groupby]
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [29]:
lr_res.sort_values('ligand_props', ascending=False).query("receptor_complex=='CD8A_CD8B' & source=='NK' & target=='B'")

,ligand,ligand_complex,ligand_logfc,ligand_means,ligand_props,ligand_zscores,mat_mean,receptor,receptor_complex,receptor_logfc,receptor_means,receptor_props,receptor_zscores,source,target,ligand_means_sums,receptor_means_sums
92,LCK,LCK,-0.016728,0.926794,0.533333,0.032533,0.199056,CD8A,CD8A_CD8B,-1.577132,0.081987,0.066667,-0.476677,NK,B,2.691222,1.490387
107,LCK,LCK,-0.016728,0.926794,0.533333,0.032533,0.199056,CD8B,CD8A_CD8B,-0.579049,0.209488,0.100000,-0.228442,NK,B,2.691222,1.199251


In [30]:
lr_res = liana_pipe(adata=adata, 
                    groupby=groupby, 
                    resource_name=resource_name,
                    expr_prop=0.1,
                    de_method=de_method,
                    base=base,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    supp_cols=None,
                    resource=None,
                    use_raw=False,
                    layer='logcounts', 
                    _return_subunits=False
                   )

/home/dbdimitrov/Repos/liana-py/liana/steady/liana_pipe.py:67: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['label'] = adata.obs[groupby]
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [32]:
lr_res.sort_values('ligand_props', ascending=False).query("receptor_complex=='CD8A_CD8B' & source=='NK' & target=='B'")

,ligand,ligand_complex,ligand_logfc,ligand_means,ligand_props,ligand_zscores,mat_mean,receptor,receptor_complex,receptor_logfc,receptor_means,receptor_props,receptor_zscores,source,target,ligand_means_sums,receptor_means_sums,prop_min


In [ ]:
from liana.steady import cellphonedb, natmi, connectome, logfc, singlecellsignalr as sca, consensus

In [ ]:
%%time
adata = consensus(adata, groupby='label', layer='logcounts', base=2, expr_prop=0.01)

In [ ]:
res = adata.uns['liana_res'].copy()

In [ ]:
res

In [ ]:
adata.uns['liana_res'].to_csv("agg.csv", index=False)

In [ ]:
res.query("receptor_complex=='HCST_KLRK1' & source=='B' & target=='B'")

Tests

In [ ]:
adata = sc.datasets.pbmc68k_reduced()

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='label', layer='logcounts')

In [ ]:
sc.get.rank_genes_groups_df(adata, 'B')

In [ ]:
labels = adata.obs.label.cat.categories

In [ ]:
# Function to get gene expr proportions
def _get_props(X_mask):
    return X_mask.getnnz(axis=0) / X_mask.shape[0]

In [ ]:
# Get Props
dedict = {}
for label in labels:
    temp = adata[adata.obs.label == label, :]
    a = _get_props(temp.X)
    dedict[label] = pd.DataFrame({'names': temp.var_names, 'props': a}).\
        assign(label=label).sort_values('names')

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='label',
                        method=de_method, use_raw=False)
# dedict = {label: sc.get.rank_genes_groups_df(adata, label).assign(
#     label=label).sort_values('names') for label in labels}

In [ ]:
{label: sc.get.rank_genes_groups_df(adata, label).assign(label=label).sort_values('names') for label in labels}

In [ ]:
dedict['B'].merge(sc.get.rank_genes_groups_df(adata, 'B'))

In [ ]:
adata.uns['rank_genes_groups']['pts']

In [ ]:
np.count_nonzero(adata[adata.obs.label=='B',:].X)

In [ ]:
adata.X

In [ ]:
sc.get.rank_genes_groups_df(adata, group='B')

In [ ]:
for imask, mask in enumerate(self.groups_masks):
    X_mask = self.X[mask]

    if self.comp_pts:
        self.pts[imask] = get_nonzeros(X_mask) / X_mask.shape[0]

In [ ]:
dedict = {label: sc.get.rank_genes_groups_df(adata, label).assign(
    label=label).sort_values('names') for label in labels}

In [ ]:
dedict

Consensus

In [ ]:
from liana.steady.scores.consensus import consensus

In [ ]:
from functools import reduce
from scipy.stats import rankdata, beta

In [ ]:
from liana.steady.aggregate import _robust_rank_aggregate, _rank_aggregate

In [ ]:
_methods = [cellphonedb, connectome, logfc, natmi, sca]

In [ ]:
adata = consensus(adata, groupby='bulk_labels',
                  use_raw=True
                  #, consensus_opts=False
                 )

In [ ]:
lr_res = adata.uns['liana_res']

In [ ]:
lr_res[lr_res['receptor_complex']=='CD8A_CD8B'].sort_values('spec_weight')

## Each Method

CellPhoneDB

In [ ]:
from liana import cellphonedb

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

In [ ]:
lr_res = adata.uns['liana_res']

In [ ]:
lr_res[lr_res['receptor_complex']=='CD8A_CD8B']

NATMI

In [ ]:
from liana import natmi

In [ ]:
adata = natmi(adata, groupby='label')

In [ ]:
adata.uns['liana_res']

SCA

In [ ]:
from liana import singlecellsignalr as sca

In [ ]:
adata = sca(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_means', ascending=True)

Connectome

In [ ]:
from liana import connectome

In [ ]:
adata = connectome(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('scaled_weight', ascending=False)

logFC

In [ ]:
from liana import logfc

In [ ]:
adata = logfc(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_logfc', ascending=False)

In [ ]:
np.exp(1)

try w/ real data

In [ ]:
adata = sc.datasets.pbmc3k_processed()
labels = adata.obs.louvain
cells = adata.obs_names

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata = adata[[x in cells for x in adata.obs_names]]

In [ ]:
adata.obs['label'] = labels

In [ ]:
sc.pp.log1p(adata)

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

## LIANA PIPE LINE BY LINE

In [ ]:
# I get this from the score object
_complex_cols = ['ligand_means', 'receptor_means']
# change to full list and move to _var
_add_cols = ['ligand', 'receptor',
             'ligand_means_sums', 'receptor_means_sums',
             'ligand_zscores', 'receptor_zscores',
             'ligand_logfoldchanges', 'receptor_logfoldchanges',
             'ligand_logfc', 'receptor_logfc'
             'mat_mean',
             ]

_key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [ ]:
from liana.utils.pre import check_adata, check_if_covered, format_vars, filter_resource
from liana.resource import select_resource
from liana.resource.select_resource import explode_complexes
from liana.steady.liana_pipe import _get_lr
from liana.utils.reassemble_complexes import reassemble_complexes
from scipy.sparse import csr_matrix
from liana.steady._permutations import get_means_perms

I need to double check if every step makes sense, as I build unit tests!!!

For example, are there any duplicated rows (source, target, ligand_complex, receptor_complex)

Double check again if correct subunit is kept, etc

In [ ]:
adata = check_adata(adata, True)

In [ ]:
# Define idents col name
adata.obs.label = adata.obs[groupby]

Load resource

In [ ]:
resource = select_resource(resource_name='consensus')

In [ ]:
# Decomplexify
resource = explode_complexes(resource)

In [ ]:
# Filter Resource
resource = filter_resource(resource, adata.var_names)

Create entities

In [ ]:
entities = np.union1d(np.unique(resource["ligand"]), np.unique(resource["receptor"]))

Check overlap between resource and adata

In [ ]:
check_if_covered(entities, adata.var_keys)

Get global mean for SCA before filtering

In [ ]:
if 'mat_mean' in _add_cols: # SHOULD BE METHOD NAME!
    adata.uns['mat_mean'] = np.mean(adata.X)

In [ ]:
# Filter to only include the relevant genes
adata = adata[:,np.intersect1d(entities, adata.var.index)]
adata

Get Stats

In [ ]:
lr_res = _get_lr(adata, resource, key_cols + _complex_cols + _add_cols, de_method)

In [ ]:
lr_res

Recomplexify

In [ ]:
# If I want to implement anything else but the min as complex policy, I would need to change this:
# temp = temp[[x=="INHBA_INHBB" for x in temp['ligand_complex']]].sort_values('receptor_complex')
# temp = temp[[x=="ACVR1B_ACVR2A" for x in temp['receptor_complex']]]

In [ ]:
lr_res = reassemble_complexes(lr_res, _key_cols, _complex_cols)

In [ ]:
lr_res

### SCORES

In [ ]:
from liana.steady.Method import Method, MethodMeta
from liana.steady.scores.cellphonedb import _cpdb_score

CELLPHONEDB

In [ ]:
_cellphonedb = MethodMeta(method_name = "cellphonedb",
                          complex_cols = ['ligand_means', 'receptor_means'], 
                          add_cols = ['ligand', 'receptor'],
                          fun = _cpdb_score,
                          magnitude = 'lr_means',
                          specificity = 'pvals',
                          permute = True,
                          reference = 'Efremova et al., 2020')

In [ ]:
cellphonedb = Method(_SCORE=_cellphonedb)

In [ ]:
adata = cellphonedb(adata, groupby, seed=666)

In [ ]:
adata.uns['liana_res']

logFC re-implement

Magnitude as None

In [ ]:
lr_res['logfc'] = lr_res[['ligand_logfoldchanges', 'receptor_logfoldchanges']].mean(1)

In [ ]:
lr_res.sort_values(by='logfc', key=abs, ascending=False)

Re-implement Connectome

Both Specificity and Magnitude

In [ ]:
lr_res['edge_weight'] = lr_res[['ligand_zscores', 'receptor_zscores']].mean(1)